In [2]:
import os
import sys

sys.path.append('..')

from polymarket_agent.get_data import get_fear_greed_by_date, get_crypto_news_from_cryptopanic, get_klines_ta, add_days
from polymarket_agent.one_shot_agent import (create_prompt, get_probability_from_openrouter,
                                             extract_yes_no_logits_and_softmax, extract_yes_no_from_text,
                                             process_results)

In [3]:
openrouter_models_non_reasoning = [
    "deepseek/deepseek-chat-v3-0324:free",
    "openai/gpt-4o-2024-11-20",    
    "openai/chatgpt-4o-latest",
]

openrouter_models_reasoning = [
    "qwen/qwq-32b:free",
    # "google/gemini-2.5-pro-exp-03-25:free",
    "google/gemini-2.5-pro-preview-03-25",
    "anthropic/claude-3.7-sonnet:thinking",
    "deepseek/deepseek-r1:free"
]

In [11]:
from tqdm import tqdm
import pandas as pd
from datetime import datetime

tiket = "BTC"
# start_day = "2025-03-01"
start_day = '2025-04-20'
end_day = "2025-04-25"
# days = range(55)
days = range(6)
dfs = []
system_prompt = "You are an expert financial analyst. You will be given a market question and relevant data. Based solely on the data provided, answer the question. Respond *only* with 'Yes' or 'No'. Do not include any reasoning, explanation, or additional text."


for d in tqdm(days):
    day = add_days(start_day, d)
    csv_path = f'openrouter_{day}.csv'
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path)
        dfs.append(df)
        continue

    end_date = add_days(start_day, d - 1)
    start_date = add_days(end_date, -14)

    news_df = get_crypto_news_from_cryptopanic(tiket, start_date, end_date)
    klines_df = get_klines_ta(tiket + 'USDT', start_date, end_date)
    greed_fear_index_df = get_fear_greed_by_date(start_date, end_date)
    question = f"Bitcoin Up or Down on date {day}"
    prompt = create_prompt(question, news_df, klines_df, greed_fear_index_df)

    responses = {}

    for model in openrouter_models_non_reasoning + openrouter_models_reasoning:
        responses[model] = get_probability_from_openrouter(
            system_prompt=system_prompt,
            prompt=prompt,
            model=model,
            temperature=0,
            max_tokens=3000,
            logprobs=True,
            top_logprobs=5,
            extra_params={
                'provider': {
                    'ignore': [
                        "Targon"
                    ]
                }
            }
        )
    df = process_results(responses, openrouter_models_non_reasoning, openrouter_models_reasoning)
    df['date'] = day
    df.to_csv(csv_path, index=False)
    dfs.append(df)

result_df = pd.concat(dfs).reset_index(drop=True)
result_df

100%|██████████| 6/6 [02:15<00:00, 22.60s/it]


,model,text,yes_tokens_found,no_tokens_found,probability_token,probability_text,prob,date
0,deepseek/deepseek-chat-v3-0324:free,no,"['Yes', 'Yes']","['No', 'No']",0.000075,0.0,0.000075,2025-04-20
1,openai/gpt-4o-2024-11-20,no,['Yes'],['No'],0.029312,0.0,0.029312,2025-04-20
2,openai/chatgpt-4o-latest,no,['Yes'],['No'],0.500000,0.0,0.500000,2025-04-20
3,qwen/qwq-32b:free,no,[],['no'],NaN,0.0,0.000000,2025-04-20
4,google/gemini-2.5-pro-preview-03-25,no,[],['no'],NaN,0.0,0.000000,2025-04-20
5,anthropic/claude-3.7-sonnet:thinking,no,[],['no'],NaN,0.0,0.000000,2025-04-20
6,deepseek/deepseek-r1:free,no,[],['no'],NaN,0.0,0.000000,2025-04-20
7,deepseek/deepseek-chat-v3-0324:free,no,"['Yes', 'Yes']","['No', 'No']",0.000028,0.0,0.000028,2025-04-21
8,openai/gpt-4o-2024-11-20,no,['Yes'],['No'],0.010987,0.0,0.010987,2025-04-21
9,openai/chatgpt-4o-latest,yes,['Yes'],['No'],0.731059,1.0,0.731059,2025-04-21
